# Production

- [Item 54: Consider Module-Scoped Code to Configure Deployment Environments](#Item-54:-Consider-Module-Scoped-Code-to-Configure-Deployment-Environments)
- [Item 55: Use repr Strings for Debugging Output](#Item-55:-Use-repr-Strings-for-Debugging-Output)
- [Item 56: Test Everything with unittest](#Item-56:-Test-Everything-with-unittest)

## Item 54: Consider Module-Scoped Code to Configure Deployment Environments

```python
# dev_main.py
TESTING = True
import db_connection
db = db_connection.Database()

# prod_main.py
TESTING = False
import db_connection
db = db_connection.Database()

# db_connection.py
import __main__

class TestingDatabase(object):
    # ...
    
class RealDatabase(object):
    # ...
    
if __main__.TESTING:
    Database = TestingDatabase
else:
    Database = RealDatabase
```

**Note:**

Once your deployment environments get complicated, you should consider moving them out of Python constants (like TESTING) and into dedicated configuration files.

```python
# db_connection.py
import sys

class Win32Database(object):
    # ...
    
class PosixDatabase(object):
    # ...
    
if sys.platform.startswith('win32'):
    Database = Win32Database
else:
    Database = PosixDatabase
```

### Things to Remember

- Programs often need to run in multiple deployment environments that each have unique assumptions and configurations.
- You can tailor a module's contents to different deployment environments by using normal Python statements in module scope.
- Module contents can be the product of any external condition, including host introspection through the *sys* and *os* modules.

## Item 55: Use *repr* Strings for Debugging Output


In [ ]:
print(repr(5))
print(repr('5'))

# equivalent
print('%r' % 5)
print('%r' % '5')

In [ ]:
class Coordinate(object):
    def __init__(self, x, y):
        self.x, self.y = x, y
        
    def __repr__(self):
        return 'Coordinate(%d, %d)' % (self.x, self.y)
    
    def __str__(self):
        return '<%d, %d>' % (self.x, self.y)
    
obj = Coordinate(1, 2)
print(repr(obj))
print(obj)
print(obj.__dict__)

Stackoverflow: [Difference between \_\_str__ and \_\_repr__?
](https://stackoverflow.com/questions/1436703/difference-between-str-and-repr)

- if `__repr__` is defined, and `__str__` is not, the object will behave as though `__str__=__repr__`.
- `__repr__` goal is to be unambiguous, while `__str__` goal is to be readable.

### Things to Remember

- Calling `print` on built-in Python types will produce the human-readable string version of a value, which hides type information
- Calling `repr` on built-in Python types will produce the printable string version of a value. These `repr` strings could be passed to the `eval` built-in function to get back thr original value.
- `%s` in format strings will produce human-readable strings like `str`. `%r` will produce printable strings like `repr`.
- You can define the `__repr__` method to customize the printable representation of a class and provide more detailed debugging information.
- You can reach into any object's `__dict__` attribute to view its internals.

## Item 56: Test Everything with unittest

> Code without tests is broken by design - Jacob Kaplan-Moss

```python
# utils.py
def to_str(data):
    if isinstance(data, str):
        return data
    elif isinstance(data, bytes):
        return data.decode('utf-8')
    else:
        raise TypeError('Must supply str or bytes, '
                        'found: %r' % data)
        
# utils_test.py
from unittest import TestCase, main
from utils import to_str

class UtilsTestCase(TestCase):
    def setUp(self):
        # ...
        
    def tearDown(self):
        # ...
    
    def test_to_str_bytes(self):
        self.assertEqual('hello', to_str(b'hello'))
       
    def test_to_str_str(self):
        self.assertEqual('hello', to_str('hello'))
    
    def test_to_str_bad(self):
        self.assertRaises(TypeError, to_str, object())
        
if __name__ == '__main__':
    main()
```

You have no guarantee that your modules will actually work together unless you prove it.

### Things to Remember

- The only way to have confidence in a Python program is to write tests.
- The `unittest` built-in module provides most of the facilities you'll need to write good tests.
- You can define tests by subclassing `TestCase` and defining one method per behavior you'd like to test. Test methods on `TestCase` classes must start with the word `test`.
- It's important to write both unit test (for isolated functionality) and integration tests (for modules that interact).